In [ ]:
import pandas as pd
import os

def limpiar_correos(df):
    patrones_invalidos = ['^sin.?correo$', '^no.?tiene.?correo$', '^sincorreo$', '^SINCORREO$']
    regex_invalido = '|'.join(patrones_invalidos)
    df = df[~df['CORREO'].str.strip().str.lower().str.match(regex_invalido, na=False)]

    df['CORREO'] = df['CORREO'].str.strip().str.lower()
    df['CORREO'] = df['CORREO'].apply(corregir_multiples_arrobas)
    df['CORREO'] = df['CORREO'].apply(corregir_dominio)
    df = df.drop_duplicates(subset=['RUT'])

    return df

def corregir_multiples_arrobas(correo):
    if pd.isnull(correo):
        return correo
    partes = correo.split('@')
    if len(partes) > 2:
        correo = partes[0] + '@' + partes[-1]
    return correo

def corregir_dominio(correo):
    if pd.isnull(correo) or '@' not in correo:
        return correo
    usuario, dominio = correo.split('@')
    if 'gmail' in dominio:
        return f'{usuario}@gmail.com'
    elif 'outlook' in dominio or 'hotmail' in dominio:
        return f'{usuario}@outlook.com'
    elif 'yahoo' in dominio:
        return f'{usuario}@yahoo.com'
    else:
        return correo

ruta_archivo = input("Ingresa la ruta completa o relativa del archivo CSV a limpiar: ").strip()

if not os.path.exists(ruta_archivo):
    print(f"⚠️ El archivo no se encuentra en la ruta: {ruta_archivo}")
else:
    df = pd.read_csv(ruta_archivo, sep=';', usecols=[0,1,2], names=['RUT', 'NOMBRE', 'CORREO'], header=0, encoding='latin1')
    df_limpio = limpiar_correos(df)
    carpeta, nombre = os.path.split(ruta_archivo)
    nombre_sin_ext = os.path.splitext(nombre)[0]
    salida = os.path.join(carpeta, f"{nombre_sin_ext}_LIMPIO.xlsx")
    df_limpio.to_excel(salida, index=False)
    print(f"✅ Archivo limpio guardado en: {salida}")
